In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
ipl1 = spark.read.format('csv').option('header', 'true').load('matches.csv')
 
ipl1.createOrReplaceTempView('matches')
 
ipl1.show()

+---+------+---------+---------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------------+-------------+-------+
| id|season|     city|     date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|             umpire1|      umpire2|umpire3|
+---+------+---------+---------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------------+-------------+-------+
|  1|  2017|Hyderabad| 4/5/2017| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|Rajiv Gandhi Inte...|         AY Dandekar

In [3]:
ipl2 = spark.read.format('csv').option('header', 'true').load('deliveries.csv')
 
ipl2.createOrReplaceTempView('deliveries')
 
ipl2.show()

+---+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------------+
| id|inning|       batting_team|        bowling_team|over|ball|     batsman| non_striker|     bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|player_dismissed|dismissal_kind|      fielder|
+---+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------------+
|  1|     1|Sunrisers Hyderabad|Royal Challengers...|   1|   1|   DA Warner|    S Dhawan|   TS Mills|            0|        0|       0|          0|          0|           0|           0|         0|         0|            null|          null|   

In [4]:
# The goal is to find player of the match from 2008 to 2019 based on season, city and venue where player of the match is virat kohli

spark.sql(' select season, player_of_match, venue, city from matches where player_of_match="V Kohli"').show()

+------+---------------+--------------------+---------+
|season|player_of_match|               venue|     city|
+------+---------------+--------------------+---------+
|  2011|        V Kohli|    Feroz Shah Kotla|    Delhi|
|  2011|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2013|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2013|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2013|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2015|        V Kohli|Rajiv Gandhi Inte...|Hyderabad|
|  2016|        V Kohli|Saurashtra Cricke...|   Rajkot|
|  2016|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2016|        V Kohli|        Eden Gardens|  Kolkata|
|  2016|        V Kohli|M Chinnaswamy Sta...|Bangalore|
|  2016|        V Kohli|Shaheed Veer Nara...|   Raipur|
|  2019|        V Kohli|        Eden Gardens|  Kolkata|
+------+---------------+--------------------+---------+



In [5]:
# To find the matches which are tie from 2008 to 2019.

spark.sql('select season, team1, team2, result from matches where result = "tie"').show()

+------+--------------------+--------------------+------+
|season|               team1|               team2|result|
+------+--------------------+--------------------+------+
|  2017|       Gujarat Lions|      Mumbai Indians|   tie|
|  2009|    Rajasthan Royals|Kolkata Knight Ri...|   tie|
|  2010|     Kings XI Punjab| Chennai Super Kings|   tie|
|  2013|Royal Challengers...| Sunrisers Hyderabad|   tie|
|  2013|    Delhi Daredevils|Royal Challengers...|   tie|
|  2014|    Rajasthan Royals|Kolkata Knight Ri...|   tie|
|  2015|    Rajasthan Royals|     Kings XI Punjab|   tie|
|  2019|Kolkata Knight Ri...|      Delhi Capitals|   tie|
|  2019|      Mumbai Indians| Sunrisers Hyderabad|   tie|
+------+--------------------+--------------------+------+



In [6]:
# To find highest number of wins by each team from 2008 to 2019 based on result.

spark.sql('select winner, count(result) as result from matches where group by winner order by winner').show()

+--------------------+------+
|              winner|result|
+--------------------+------+
|                null|     4|
| Chennai Super Kings|   100|
|     Deccan Chargers|    29|
|      Delhi Capitals|    10|
|    Delhi Daredevils|    67|
|       Gujarat Lions|    13|
|     Kings XI Punjab|    82|
|Kochi Tuskers Kerala|     6|
|Kolkata Knight Ri...|    92|
|      Mumbai Indians|   109|
|       Pune Warriors|    12|
|    Rajasthan Royals|    75|
|Rising Pune Super...|    10|
|Rising Pune Super...|     5|
|Royal Challengers...|    84|
| Sunrisers Hyderabad|    58|
+--------------------+------+



In [7]:
# To find the winning team with 100+ margin of runs based on season, toss winner and player of the match

spark.sql('select season, winner, toss_winner, win_by_runs, player_of_match from matches where win_by_runs > 100').show()

+------+--------------------+--------------------+-----------+---------------+
|season|              winner|         toss_winner|win_by_runs|player_of_match|
+------+--------------------+--------------------+-----------+---------------+
|  2017|      Mumbai Indians|    Delhi Daredevils|        146|    LMP Simmons|
|  2008|Kolkata Knight Ri...|Royal Challengers...|        140|    BB McCullum|
|  2008|    Rajasthan Royals|    Delhi Daredevils|        105|      SR Watson|
|  2011|     Kings XI Punjab|     Kings XI Punjab|        111|   AC Gilchrist|
|  2013|Royal Challengers...|       Pune Warriors|        130|       CH Gayle|
|  2015|Royal Challengers...|     Kings XI Punjab|        138|       CH Gayle|
|  2016|Royal Challengers...|       Gujarat Lions|        144| AB de Villiers|
|  2018|      Mumbai Indians|Kolkata Knight Ri...|        102|   Ishan Kishan|
|  2019| Sunrisers Hyderabad|Royal Challengers...|        118|     J Bairstow|
+------+--------------------+--------------------+--

In [8]:
# To find the team which win by 10 wickets margin based on season, winner, toss_winner, toss_decision, venue and player of the match.

spark.sql('select season, winner, toss_winner, toss_decision, venue, player_of_match from matches where win_by_wickets >= 10').show()

+------+--------------------+--------------------+-------------+--------------------+---------------+
|season|              winner|         toss_winner|toss_decision|               venue|player_of_match|
+------+--------------------+--------------------+-------------+--------------------+---------------+
|  2017|Kolkata Knight Ri...|Kolkata Knight Ri...|        field|Saurashtra Cricke...|        CA Lynn|
|  2017|     Kings XI Punjab|     Kings XI Punjab|        field|Punjab Cricket As...| Sandeep Sharma|
|  2008|     Deccan Chargers|     Deccan Chargers|        field|Dr DY Patil Sport...|   AC Gilchrist|
|  2009|    Delhi Daredevils|    Delhi Daredevils|        field|            Newlands|     DL Vettori|
|  2010|Royal Challengers...|Royal Challengers...|        field|M Chinnaswamy Sta...|      JH Kallis|
|  2011|    Rajasthan Royals|      Mumbai Indians|          bat|    Wankhede Stadium|      SR Watson|
|  2012|      Mumbai Indians|    Rajasthan Royals|          bat|Sawai Mansingh St.

In [15]:
# Goal to combine two tables using Join where winner and player of the match from matches table and batsman, non striker and is super over from deliveries table.

spark.sql('select matches.winner, matches.player_of_match, deliveries.batsman, deliveries.non_striker, deliveries.is_super_over = 1 FROM matches JOIN deliveries where matches.id = deliveries.id').show()

+--------------------+---------------+------------+------------+-------------------+
|              winner|player_of_match|     batsman| non_striker|(is_super_over = 1)|
+--------------------+---------------+------------+------------+-------------------+
| Sunrisers Hyderabad|   Yuvraj Singh|   DA Warner|    S Dhawan|              false|
|Rising Pune Super...|      SPD Smith|   DA Warner|    S Dhawan|              false|
|Kolkata Knight Ri...|        CA Lynn|   DA Warner|    S Dhawan|              false|
|     Kings XI Punjab|     GJ Maxwell|   DA Warner|    S Dhawan|              false|
|Royal Challengers...|      KM Jadhav|   DA Warner|    S Dhawan|              false|
| Sunrisers Hyderabad|    Rashid Khan|    S Dhawan|   DA Warner|              false|
|      Mumbai Indians|         N Rana|    S Dhawan|   DA Warner|              false|
|     Kings XI Punjab|       AR Patel|    S Dhawan|   DA Warner|              false|
|    Delhi Daredevils|      SV Samson|   DA Warner|    S Dhawan| 